In [1]:
!pip install -q openai

In [ ]:
from IPython.display import Markdown, display
from openai import OpenAI
import os

openai_api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {role: "system", "content":"You are a great philosopher."}
        {role: "user", "content":"What is the meaning of life?"}
    ]
)
display(Markdown(response.choices[0].message.content))

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

file_path = "Reddit_Title.csv"
data = pd.read_csv(file_path, sep=";")

data_cleaned = data[['title','label']].head(200)

label_mapping = {0: "non-stress", 1:"stress"}
data_cleaned['label'] = data_cleaned['label'].map(label_mapping)

train_data, validation_data = train_test_split(data_cleaned, test_size=0.2, random_state=42)

def save_to_jsonl(data, output_file_path):
  jsonl_data = []
  for index, row in data.iterrows():
    jsonl_data.append({
        "messages": [
            {"role": "system", "content":"Given a social media post, classify whether it indicates 'stress' or 'non-stress'."},
            {"role": "user", "content":row['title']},
            {"role": "assistant", "content":f"\"{row['label']}\""}
        ]
    })

    with open(output_file_path, 'w') as f:
      for item in jsonl_data:
        f.write(json.dumps(item)+'\n')

train_output_file_path = 'stress_detection_train.jsonl'
validation_output_file_path = 'stress_detection_validation.jsonl'

save_to_jsonl(train_data, train_output_file_path)
save_to_jsonl(validation_data, validation_output_file_path)

print(f"Training dataset save to {train_output_file_path}")
print(f"Validation dataset save to {validation_output_file_path}")

In [ ]:
train_file = client.files.create(
    file=open(train_output_file_path, "rb")
    purpose="fine-tune"
)

validation_file = client.files.create(
    file=open(validation_output_file_path, "rb")
    purpose="fine-tune"
)

In [ ]:
model = client.fine_tuning.jobs.create(
    training_file = train_file.id,
    validation_file = valid_file.id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={
    "n_epochs": 3,
    "batch_size": 3,
    "learning_rate_multiplier": 0.3
  }
)

job_id = model.id
status = model.status

print(f'Fine-tuning model with jobID: {job_id}.')
print(f"Training Response: {model}")
print(f"Training Status: {status}")